In [1]:
from datetime import datetime
import os, sys
import logging
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, TextStreamer, BitsAndBytesConfig

In [2]:
# MODEL = '/home/cdsw/models/self_rag_critic'
# MODEL = '/home/cdsw/models/selfrag_llama2_7b'
MODEL = '/home/cdsw/models/Mistral-7B-Instruct-v0.2'

tokenizer = AutoTokenizer.from_pretrained(MODEL, model_max_length=8192)
tokenizer.pad_token = tokenizer.eos_token

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [3]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=False,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(
    MODEL,
    quantization_config=bnb_config,
    use_cache=True, # False when training
    # do_sample=True,
    device_map='auto'
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [4]:
gen_config = model.generation_config
gen_config.pad_token_id=tokenizer.eos_token_id
gen_config

GenerationConfig {
  "bos_token_id": 1,
  "do_sample": true,
  "eos_token_id": 2,
  "max_length": 4096,
  "pad_token_id": 2,
  "temperature": 0.6,
  "top_p": 0.9
}

In [14]:
PROMPT_DICT = {
    "context": (
        "Given an instruction, please make a judgment on whether finding some external documents from the web (e.g., Wikipedia) helps to generate a better response. Please answer [Yes] or [No] and write an explanation.\n\n"
        "##\nInstruction: Give three tips for staying healthy.\n"
        "Need retrieval?: [Yes]\n"
        "Explanation: There might be some online sources listing three tips for staying healthy or some reliable sources to explain the effects of different behaviors on health. So retrieving documents is helpful to improve the response to this query.\n\n"
        "##\nInstruction: Describe a time when you had to make a difficult decision.\n"
        "Need retrieval?: [No]\n"
        "Explanation: This instruction is asking about some personal experience and thus it does not require one to find some external documents.\n\n"
        "##\nInstruction: Write a short story in third person narration about a protagonist who has to make an important career decision.\n"
        "Need retrieval?: [No]\n"
        "Explanation: This instruction asks us to write a short story, which does not require external evidence to verify.\n\n"
        "##\nInstruction: What is the capital of France?\n"
        "Need retrieval?: [Yes]\n"
        "Explanation: While the instruction simply asks us to answer the capital of France, which is a widely known fact, retrieving web documents for this question can still help.\n\n"
        "##\n Instruction: Find the area of a circle given its radius. Radius = 4\n"
        "Need retrieval?: [No]\n"
        "Explanation: This is a math question and although we may be able to find some documents describing a formula, it is unlikely to find a document exactly mentioning the answer.\n\n"
        "##\nInstruction: Arrange the words in the given sentence to form a grammatically correct sentence. quickly the brown fox jumped\n"
        "Need retrieval?: [No]\n"
        "Explanation: This task doesn't require any external evidence, as it is a simple grammatical question.\n\n"
        "##\nInstruction: Explain the process of cellular respiration in plants."
        "Need retrieval?: [Yes]\n"
        "Explanation: This instruction asks for a detailed description of a scientific concept, and is highly likely that we can find a reliable and useful document to support the response.\n\n"
        "##\nInstruction:{instruction}\n"
        "Need retrieval?: "
    ),
    "multi_retrieval": (
        "You will be provided with an instruction, evidence, output sentence, and preceding sentences (optional). If the preceding sentence is given, the output should be the sentence that follows those preceding sentences.  Your task is to determine whether the information in the output sentence can be fully verified by the evidence or if it requires further external verification. If the output sentence can be verified solely with the evidence or doesn’t require any verification, respond with [No Retrieval]. If additional information is needed to verify the output sentence, respond with [Retrieval]. Please provide explanations for your judgments.\n\n"
        "##\nInstruction: Explain the use of word embeddings in Natural Language Processing.\n"
        "Preceding sentences: Word embeddings are one of the most powerful tools available for Natural Language Processing (NLP). They are mathematical representations of words or phrases in a vector space, allowing similarities between words and the context in which they are used to be measured.\n"
        "Evidence: Word embedding\nWord embedding is the collective name for a set of language modeling and feature learning techniques in natural language processing (NLP) where words or phrases from the vocabulary are mapped to vectors of real numbers. Conceptually it involves a mathematical embedding from a space with one dimension per word to a continuous vector space with a much lower dimension.\n"
        "Output: Word embeddings are useful for tasks such as sentiment analysis, text classification, predicting the next word in a sequence, and understanding synonyms and analogies.\n"
        "Rating: [Retrieval]\n"
        "Explanation: The output discusses the applications of word embeddings, while the evidence only discusses the definitions of word embeddings and how it works. Therefore, we need to retrieve other evidence to verify whether the output is actually correct or not.\n"
        "###\nInstruction: {instruction}\n"
        "Preceding sentences: {preceding_sentences}\n"
        "Evidence: {evidence}\n"
        "Output: {target_output}\n"
        "Rating: "),
    "multi_retrieval_no_preceding": (
        "You will be provided with an instruction, evidence, output sentence, and preceding sentences (optional). If the preceding sentence is given, the output should be the sentence that follows those preceding sentences.  Your task is to determine whether the information in the output sentence can be fully verified by the evidence or if it requires further external verification. If the output sentence can be verified solely with the evidence or doesn’t require any verification, respond with [No Retrieval]. If additional information is needed to verify the output sentence, respond with [Retrieval]. Please provide explanations for your judgments.\n"
        "\n##\nInstruction: Explain the use of word embeddings in Natural Language Processing.\n"
        "Evidence: Word embedding\nWord embedding is the collective name for a set of language modeling and feature learning techniques in natural language processing (NLP) where words or phrases from the vocabulary are mapped to vectors of real numbers. Conceptually it involves a mathematical embedding from a space with one dimension per word to a continuous vector space with a much lower dimension.\n"
        "Output: Word embeddings are useful for tasks such as sentiment analysis, text classification, predicting the next word in a sequence, and understanding synonyms and analogies.\n"
        "Rating: [Retrieval]\n"
        "Explanation: The output discusses the applications of word embeddings, while the evidence only discusses the definitions of word embeddings and how it works. Therefore, we need to retrieve other evidence to verify whether the output is actually correct or not.\n"
        "\n##\nInstruction: {instruction}\n"
        "Evidence: {evidence}\n"
        "Output: {target_output}\n"
        "Rating: "
    ),
    "multi_retrieval_three_way": (
        "You will be provided with an instruction, evidence, output sentence, and preceding sentences (optional). If the preceding sentence is given, the output should be the sentence that follows those preceding sentences.  Your task is to determine whether the information in the output sentence can be fully verified by the evidence or if it requires further external verification. There are three cases:\n"
        "- If the output sentence can be verified solely with the evidence, then respond with [Continue to Use Evidence]. \n"
        "- If the sentence doesn't require any factual verification (e.g., a subjective sentence or a sentence about common sense), then respond with  [No Retrieval]. \n"
        "If additional information is needed to verify the output sentence, respond with [Retrieval]. Please provide explanations for your judgments. \n\n"
        "##\nInstruction: Explain the use of word embeddings in Natural Language Processing.\n"
        "Preceding sentences: Word embeddings are one of the most powerful tools available for Natural Language Processing (NLP). They are mathematical representations of words or phrases in a vector space, allowing similarities between words and the context in which they are used to be measured. \n"
        "Evidence:\nWord embedding\nWord embedding is the collective name for a set of language modeling and feature learning techniques in natural language processing (NLP) where words or phrases from the vocabulary are mapped to vectors of real numbers. Conceptually it involves a mathematical embedding from a space with one dimension per word to a continuous vector space with a much lower dimension. \n"
        "Output: Word embeddings are useful for tasks such as sentiment analysis, text classification, predicting the next word in a sequence, and understanding synonyms and analogies.\n"
        "Rating: [Retrieval]\n"
        "Explanation: The output discusses the applications of word embeddings, while the evidence only discusses the definitions of word embeddings and how it works. Therefore, we need to retrieve other evidence to verify whether the output is correct or not.\n"
        "###\nInstruction: {instruction}\n"
        "Preceding sentences: {preceding_sentences}\n"
        "Evidence: {evidence}\n"
        "Output: {target_output}\n"
        "Rating: "
    ),
    "multi_retrieval_three_way_no_preceding": (
        "You will be provided with an instruction, evidence, output sentence, and preceding sentences (optional). If the preceding sentence is given, the output should be the sentence that follows those preceding sentences.  Your task is to determine whether the information in the output sentence can be fully verified by the evidence or if it requires further external verification. There are three cases:\n"
        "- If the output sentence can be verified solely with the evidence, then respond with [Continue to Use Evidence]. \n"
        "- If the sentence doesn't require any factual verification (e.g., a subjective sentence or a sentence about common sense), then respond with  [No Retrieval]. \n"
        "- If additional information is needed to verify the output sentence, respond with [Retrieval]. Please provide explanations for your judgments. \n\n"
        "##\nInstruction: Explain the use of word embeddings in Natural Language Processing.\n"
        "Preceding sentences: Word embeddings are one of the most powerful tools available for Natural Language Processing (NLP). They are mathematical representations of words or phrases in a vector space, allowing similarities between words and the context in which they are used to be measured. \n"
        "Evidence:\nWord embedding\nWord embedding is the collective name for a set of language modeling and feature learning techniques in natural language processing (NLP) where words or phrases from the vocabulary are mapped to vectors of real numbers. Conceptually it involves a mathematical embedding from a space with one dimension per word to a continuous vector space with a much lower dimension. \n"
        "Output: Word embeddings are useful for tasks such as sentiment analysis, text classification, predicting the next word in a sequence, and understanding synonyms and analogies.\n"
        "Rating: [Retrieval]\n"
        "Explanation: The output discusses the applications of word embeddings, while the evidence only discusses the definitions of word embeddings and how it works. Therefore, we need to retrieve other evidence to verify whether the output is correct or not.\n"
        "###\nInstruction: {instruction}\n"
        "Evidence: {evidence}\n"
        "Output: {target_output}\n"
        "Rating: "
    )
}

In [6]:
def generate(prompt, max_new_tokens=200):
    print(prompt)
    print('\nTokenizing ...\n')
    input_ids = tokenizer([prompt], return_tensors="pt", truncation=True).to('cuda')
    streamer = TextStreamer(tokenizer, skip_prompt=True)
    print('Start inference:', str(datetime.now()))
    _ = model.generate(
        **input_ids,
        streamer=streamer,
        max_new_tokens=max_new_tokens,
        do_sample=False,
        # top_p=0.9,
        # temperature=0.1,
        # early_stopping=True,
        generation_config=gen_config,
    )
    print('Completed:', str(datetime.now()))

In [7]:
query = 'What is the difference between Cloudera CDP Base 7.1.7 and 7.1.9?'
paragraph = '''What's new in Cloudera Runtime 7.1.9
Understand the functionalities and improvements to features of components in Cloudera Runtime 7.1.9.
Open Data Lakehouse, powered by Apache Iceberg
CDP Private Cloud Base 7.1.9 delivers the hybrid Open Data Lakehouse providing the following benefits:
Open architecture
Cloudera’s Open Data Lakehouse, powered by Apache Iceberg is 100% open—open source, open standards based, and with wide community adoption. It can store multiple data formats and enables multiple engines to work on the same data.
Ease of adoption
By integrating Iceberg right into the Shared Data Experience (SDX) and Apache Ozone, Cloudera offers the easiest path to deploying a lakehouse. Additional capabilities like schema evolution, hidden partition, and more simplify data management for large datasets.'''

In [8]:
prompt = PROMPT_DICT['context'].format_map({
    'instruction': query
})
generate(prompt, max_new_tokens=5)

Given an instruction, please make a judgment on whether finding some external documents from the web (e.g., Wikipedia) helps to generate a better response. Please answer [Yes] or [No] and write an explanation.

##
Instruction: Give three tips for staying healthy.
Need retrieval?: [Yes]
Explanation: There might be some online sources listing three tips for staying healthy or some reliable sources to explain the effects of different behaviors on health. So retrieving documents is helpful to improve the response to this query.

##
Instruction: Describe a time when you had to make a difficult decision.
Need retrieval?: [No]
Explanation: This instruction is asking about some personal experience and thus it does not require one to find some external documents.

##
Instruction: Write a short story in third person narration about a protagonist who has to make an important career decision.
Need retrieval?: [No]
Explanation: This instruction asks us to write a short story, which does not require

/home/cdsw/.local/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:492: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/home/cdsw/.local/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:497: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


[Yes]
Ex
Completed: 2024-04-17 09:36:18.609312


In [12]:
query = "Can you tell me the difference between llamas and alpacas?",
paragraph = "The alpaca (Lama pacos) is a species of South American camelid mammal. It is similar to, and often confused with, the llama. Alpacas are considerably smaller than llamas, and unlike llamas, they were not bred to be working animals, but were bred specifically for their fiber."

In [15]:
prompt = PROMPT_DICT['multi_retrieval_no_preceding'].format_map({
    'instruction': query,
    'evidence': paragraph,
    'target_output': 'The main difference between Cloudera CDP Base 7.1.7 and 7.1.9 is the inclusion of Apache Iceberg in the 7.1.9 release.'
})
generate(prompt, max_new_tokens=5)

You will be provided with an instruction, evidence, output sentence, and preceding sentences (optional). If the preceding sentence is given, the output should be the sentence that follows those preceding sentences.  Your task is to determine whether the information in the output sentence can be fully verified by the evidence or if it requires further external verification. If the output sentence can be verified solely with the evidence or doesn’t require any verification, respond with [No Retrieval]. If additional information is needed to verify the output sentence, respond with [Retrieval]. Please provide explanations for your judgments.

##
Instruction: Explain the use of word embeddings in Natural Language Processing.
Evidence: Word embedding
Word embedding is the collective name for a set of language modeling and feature learning techniques in natural language processing (NLP) where words or phrases from the vocabulary are mapped to vectors of real numbers. Conceptually it involves